In [24]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='dataset/Cora', name='Cora')#包括数据集的下载，若root路径存在数据集则直接加载数据集
data = dataset[0] #该数据集只有一个图len(dataset)：1
# Data(edge_index=[2, 10556], test_mask=[2708],
#         train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

In [25]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
#设计Net
class MLP(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(MLP, self).__init__()
        torch.manual_seed(12345) #设定随机种子，可省略
        self.lin1 = Linear(dataset.num_features, hidden_channels)
        self.lin2 = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x):
        x = self.lin1(x)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

model = MLP(hidden_channels=16)
print(model)
#MLP(
#  (lin1): Linear(in_features=1433, out_features=16, bias=True)
#  (lin2): Linear(in_features=16, out_features=7, bias=True)
#)

MLP(
  (lin1): Linear(in_features=1433, out_features=16, bias=True)
  (lin2): Linear(in_features=16, out_features=7, bias=True)
)


In [26]:
import torch
from torch_geometric.datasets import Planetoid
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

#设计Net
class GCN(torch.nn.Module):
    #初始化
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)
	#前向传播
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        #注意这里输出的是节点的特征，维度为[节点数,类别数]
        return x
#实例化模型
model = GCN(hidden_channels=16)
print(model)
#GCN(
#  (conv1): GCNConv(1433, 16)
#  (conv2): GCNConv(16, 7)
#)

GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)


In [27]:
criterion = torch.nn.CrossEntropyLoss()

In [28]:
#选择优化器
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [29]:
#训练函数
def train(model,data,optimizer,criterion):
    model.train()
    optimizer.zero_grad()  # 梯度置零
    out = model(data.x, data.edge_index)  # 模型前向传播
    loss = criterion(out[data.train_mask],data.y[data.train_mask])  # 计算loss
    loss.backward()  # 反向传播
    optimizer.step()  # 优化器梯度下降
    return loss
#训练
for epoch in range(1, 201):
    loss = train(model,data,optimizer,criterion)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 001, Loss: 1.9474
Epoch: 002, Loss: 1.8486
Epoch: 003, Loss: 1.7382
Epoch: 004, Loss: 1.6300
Epoch: 005, Loss: 1.4891
Epoch: 006, Loss: 1.3653
Epoch: 007, Loss: 1.2619
Epoch: 008, Loss: 1.1165
Epoch: 009, Loss: 1.0073
Epoch: 010, Loss: 0.9298
Epoch: 011, Loss: 0.8412
Epoch: 012, Loss: 0.7436
Epoch: 013, Loss: 0.6744
Epoch: 014, Loss: 0.5305
Epoch: 015, Loss: 0.5117
Epoch: 016, Loss: 0.4814
Epoch: 017, Loss: 0.3988
Epoch: 018, Loss: 0.3728
Epoch: 019, Loss: 0.3528
Epoch: 020, Loss: 0.3245
Epoch: 021, Loss: 0.2602
Epoch: 022, Loss: 0.2214
Epoch: 023, Loss: 0.2242
Epoch: 024, Loss: 0.2133
Epoch: 025, Loss: 0.1590
Epoch: 026, Loss: 0.1755
Epoch: 027, Loss: 0.1547
Epoch: 028, Loss: 0.1346
Epoch: 029, Loss: 0.1473
Epoch: 030, Loss: 0.1218
Epoch: 031, Loss: 0.1195
Epoch: 032, Loss: 0.1266
Epoch: 033, Loss: 0.1104
Epoch: 034, Loss: 0.1113
Epoch: 035, Loss: 0.1001
Epoch: 036, Loss: 0.0929
Epoch: 037, Loss: 0.0669
Epoch: 038, Loss: 0.0808
Epoch: 039, Loss: 0.0702
Epoch: 040, Loss: 0.0994


In [30]:
#测试函数
def test(model,data):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)  # 使用最大概率的类别作为预测结果
    test_correct = pred[data.test_mask] == data.y[data.test_mask]  # 获取正确标记的节点
    test_acc = int(test_correct.sum()) / int(data.test_mask.sum()) # 计算正确率
    return test_acc
#精度评价
test_acc = test(model,data)
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.8020
